## Used Algorithms in Pandas for Data Scientists

from the internet

In [1]:
import pandas as pd
import numpy as np
np.random.randint(200, size=(20))

array([ 97,  21, 123, 157, 199,  70,  74,  85, 167, 122, 155,  96, 109,
        65, 174, 102, 198, 184,  78,   0])

# Convert in dataframe 

In [2]:
l = np.random.randint(200, size=(20))
df = pd.DataFrame(l, columns = ['Numbers'])
df

,Numbers
0,112
1,28
2,73
3,119
4,151
5,34
6,192
7,129
8,1
9,99


# Conver into categorical with bins
Use cut when you need to segment and sort data values into bins. This function is also useful for going from a continuous variable to a categorical variable. For example, cut could convert ages to groups of age ranges. Supports binning into an equal number of bins, or a pre-specified array of bins.

In [3]:
df['CutLabels'] = pd.cut(df['Numbers'], 
                         bins = [0, 33, 66, 100, 200], 
                         labels = ['Low','Medium','High', 'alto']
                        )
df

,Numbers,CutLabels
0,112,alto
1,28,Low
2,73,High
3,119,alto
4,151,alto
5,34,Medium
6,192,alto
7,129,alto
8,1,Low
9,99,High


# 2. Append a Row to a Category
I added the number 3 to the category High on the last position

In [4]:

df.loc[len(df)] = [3, 'High']
df

,Numbers,CutLabels
0,112,alto
1,28,Low
2,73,High
3,119,alto
4,151,alto
5,34,Medium
6,192,alto
7,129,alto
8,1,Low
9,99,High


# 3. Count Group Elements
I added a column that contains the count of elements in each category
New column that makes counts using the CUTLABELS values.

In [5]:
df['GroupCount'] = df.groupby(['CutLabels'])['Numbers'].transform('count')
df

,Numbers,CutLabels,GroupCount
0,112,alto,8
1,28,Low,7
2,73,High,3
3,119,alto,8
4,151,alto,8
5,34,Medium,3
6,192,alto,8
7,129,alto,8
8,1,Low,7
9,99,High,3


# 4. Get the Sum of Elements in Each Category
GroupSum column that contains the sum of each category is added.
Grouping by CutLabels and Numbers in additional

In [6]:
df['GroupSum'] = df.groupby(['CutLabels'])['Numbers'].transform(sum)
df.sort_values(by = ['CutLabels'])

,Numbers,CutLabels,GroupCount,GroupSum
20,3,High,3,175
2,73,High,3,175
9,99,High,3,175
18,16,Low,7,111
14,5,Low,7,111
12,20,Low,7,111
11,17,Low,7,111
8,1,Low,7,111
10,24,Low,7,111
1,28,Low,7,111


# 5. Replace Outliers with Median in Each Group (FAIL)
I write a function for this purpose.

In [7]:
def replace_outliers_with_median(df, col_name, new_col):
# Outlier bounds are calculated (mean -+ 2 std)
    m = np.mean(df[col_name])
    s = np.std(df[col_name])
    lower = m - 2 * s
    upper = m + 2 * s
# Replacing outliers with median
    df[new_col] = np.where(df[col_name].between(lower, upper),
                            df[col_name], df[col_name].median())
    df[new_col] = df[new_col].astype(int)
    
    

You can re-write this function to replace it with mean or to change lower and upper bounds.
I checked the outliers within each group.

df = df.groupby('CutLabels').apply(replace_outliers_with_median, col_name='Numbers', new_col='NumbersNoOutlier')
df.sort_values(by = ['CutLabels'])

# 6. Rank Numbers within Each Category
The numbers are ranked from smallest to largest under the Rank column.
It create an aditional column *rank* that is used to set the rank of the number by group of CutLabels

* note Nan CutLabels is not ranked because the cutlabel is nan also


In [8]:
df['Rank'] = df.groupby(['CutLabels'], sort=False)['Numbers'].rank()
df.sort_values(by = ['CutLabels', 'Rank'])

,Numbers,CutLabels,GroupCount,GroupSum,Rank
20,3,High,3,175,1.0
2,73,High,3,175,2.0
9,99,High,3,175,3.0
8,1,Low,7,111,1.0
14,5,Low,7,111,2.0
18,16,Low,7,111,3.0
11,17,Low,7,111,4.0
12,20,Low,7,111,5.0
10,24,Low,7,111,6.0
1,28,Low,7,111,7.0


# 7. Subtract Rows of Different Categories from Each Other According to Their Rank - Fail
I wrote a function for this too.

In [9]:
def subtract_rows(df, value_col, category_column, group_name1, group_name2, group_name3, common_columns):
    i1 = df.loc[(df[category_column] == group_name1), common_columns + [value_col]]
    i2 = df.loc[(df[category_column] == group_name2), common_columns + [value_col]].rename(columns={value_col: value_col + str(2)})
    i1 = i1.merge(i2, on=common_columns)



    
    i1[category_column] = group_name3
    i1[value_col] = i1[value_col] - i1[value_col + str(2)]
    
    i1.drop(columns=value_col + str(2), inplace=True)
    df = df.append(i1)
    
    

alling the function, you need to specify;
val_col: The numbers that are used to subtract.
category_column: The name of the column that contains category names.
group_name1: The first group of subtraction.
group_name2: The second group of subtraction.
group_name3: The result of subtraction.
common_columns: The columns that will remain the same as other rows used in subtraction.
So, I subtract mediums from lows since they have the same number of elements.

df = subtract_rows(df,'Numbers', 'CutLabels','Medium', 'Low',  'Medium-Low', ['GroupCount','Rank'])
df.sort_values(by = ['CutLabels', 'Rank'])

# 8. Sum Numbers Cumulatively within Each Group
Number in the column Numbers are summed cumulatively.
Here, it is important to sort numbers before summing cumulatively. Because they are summed in the order they are.
Nan values are not summed

In [10]:
df = df.sort_values(by = ['CutLabels', 'Rank'])
df['CumulativeSum'] = df.groupby('CutLabels')['Numbers'].transform(pd.Series.cumsum)
df.sort_values(by = ['CutLabels', 'Rank'])

,Numbers,CutLabels,GroupCount,GroupSum,Rank,CumulativeSum
20,3,High,3,175,1.0,3
2,73,High,3,175,2.0,76
9,99,High,3,175,3.0,175
8,1,Low,7,111,1.0,1
14,5,Low,7,111,2.0,6
18,16,Low,7,111,3.0,22
11,17,Low,7,111,4.0,39
12,20,Low,7,111,5.0,59
10,24,Low,7,111,6.0,83
1,28,Low,7,111,7.0,111


# 9. Convert Cumulative Data to Separated Data
From the CumulativeSum column, the DiffFromCumulative column is obtained which is the same as the GroupCount column.

In [11]:
df['DiffFromCumulative'] = df.groupby('CutLabels')['CumulativeSum'].diff(1).fillna(df['GroupCount'])
df.sort_values(by = ['CutLabels', 'Rank'])

,Numbers,CutLabels,GroupCount,GroupSum,Rank,CumulativeSum,DiffFromCumulative
20,3,High,3,175,1.0,3,3.0
2,73,High,3,175,2.0,76,73.0
9,99,High,3,175,3.0,175,99.0
8,1,Low,7,111,1.0,1,7.0
14,5,Low,7,111,2.0,6,5.0
18,16,Low,7,111,3.0,22,16.0
11,17,Low,7,111,4.0,39,17.0
12,20,Low,7,111,5.0,59,20.0
10,24,Low,7,111,6.0,83,24.0
1,28,Low,7,111,7.0,111,28.0


# 10. Convert All Columns to Int Except One Column

After some operations, we have some float columns. Let’s convert all to int but we have an object column that we can’t convert.

In [12]:
#rellenar Rank con NaN con 0
df["CutLabels"] = df["CutLabels"].fillna('Desconocido')
df["Rank"] = df["Rank"].fillna(0)
df = df.fillna(0)
df

,Numbers,CutLabels,GroupCount,GroupSum,Rank,CumulativeSum,DiffFromCumulative
20,3,High,3,175,1.0,3,3.0
2,73,High,3,175,2.0,76,73.0
9,99,High,3,175,3.0,175,99.0
8,1,Low,7,111,1.0,1,7.0
14,5,Low,7,111,2.0,6,5.0
18,16,Low,7,111,3.0,22,16.0
11,17,Low,7,111,4.0,39,17.0
12,20,Low,7,111,5.0,59,20.0
10,24,Low,7,111,6.0,83,24.0
1,28,Low,7,111,7.0,111,28.0


In [13]:

df.loc[:, df.columns != 'CutLabels'] = df.loc[:, df.columns != 'CutLabels'].astype(int)
df.sort_values(by = ['CutLabels', 'Rank'])

,Numbers,CutLabels,GroupCount,GroupSum,Rank,CumulativeSum,DiffFromCumulative
20,3,High,3,175,1.0,3,3.0
2,73,High,3,175,2.0,76,73.0
9,99,High,3,175,3.0,175,99.0
8,1,Low,7,111,1.0,1,7.0
14,5,Low,7,111,2.0,6,5.0
18,16,Low,7,111,3.0,22,16.0
11,17,Low,7,111,4.0,39,17.0
12,20,Low,7,111,5.0,59,20.0
10,24,Low,7,111,6.0,83,24.0
1,28,Low,7,111,7.0,111,28.0


# 11. Convert All Columns to Float Except Some Columns
Let’s say we want our values to be float except for some columns.

In [14]:
columns_to_convert_float = list(set(list(df.columns)) - set(['CutLabels', 'GroupCount', 'Rank']))
df[columns_to_convert_float] = df[columns_to_convert_float].astype(float)
df.sort_values(by = ['CutLabels', 'Rank'])

,Numbers,CutLabels,GroupCount,GroupSum,Rank,CumulativeSum,DiffFromCumulative
20,3.0,High,3,175.0,1.0,3.0,3.0
2,73.0,High,3,175.0,2.0,76.0,73.0
9,99.0,High,3,175.0,3.0,175.0,99.0
8,1.0,Low,7,111.0,1.0,1.0,7.0
14,5.0,Low,7,111.0,2.0,6.0,5.0
18,16.0,Low,7,111.0,3.0,22.0,16.0
11,17.0,Low,7,111.0,4.0,39.0,17.0
12,20.0,Low,7,111.0,5.0,59.0,20.0
10,24.0,Low,7,111.0,6.0,83.0,24.0
1,28.0,Low,7,111.0,7.0,111.0,28.0


# 12. Convert Rows to Columns
I converted the category names under the CutLabels column to column names.

In [ ]:
df.drop_duplicates()
df_pivot = df.pivot(index = ['Rank'], columns=['CutLabels'], values = ['Numbers'])
df_pivot.columns = df_pivot.columns.droplevel()
df_pivot = df_pivot.rename_axis(None,axis=1).reset_index()
df_pivot

# 13. Convert Columns to Rows
This time, the GroupCount and the GroupSum columns are converted to rows from the original dataframe.

In [18]:
df_melt=pd.melt(df, id_vars=['CutLabels'],value_vars=['CumulativeSum','DiffFromCumulative'],var_name='OperationName',value_name='OperationResult')
df_melt = df_melt.drop_duplicates()
df_melt

,CutLabels,OperationName,OperationResult
0,High,CumulativeSum,3.0
1,High,CumulativeSum,76.0
2,High,CumulativeSum,175.0
3,Low,CumulativeSum,1.0
4,Low,CumulativeSum,6.0
5,Low,CumulativeSum,22.0
6,Low,CumulativeSum,39.0
7,Low,CumulativeSum,59.0
8,Low,CumulativeSum,83.0
9,Low,CumulativeSum,111.0
